<center><h1>AI in Web Development</h1></center>

---

<center><h2>Lesson 02</h2></center>


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/snsie/ai-webdev/blob/main/lessons/lesson-02/lesson-02.ipynb)

##Resources used for this lesson:

1. [Backpropagation Example](https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/)

<!-- 2. [Colab Example](https://colab.research.google.com/github/casperbh96/Neural-Network-From-Scratch/blob/master/NN_From_Scratch.ipynb) -->

<center><img src='https://github.com/snsie/ai-webdev/blob/main/images/matt_network_numbered.png?raw=true'/>
<p>Parameterized Network</p></center>

---

<br/>

\begin{align}
\textrm{Inputs: }
\left[
\begin{matrix}
0.05 & 0.10
\end{matrix}
\right]
\end{align}

<br/>

\begin{align}
\textrm{Outputs: }
\left[
\begin{matrix}
0.01 & 0.99
\end{matrix}
\right]
\end{align}

<br/>

\begin{align}
\textrm{Weights to Hidden Layer: }
\left[
\begin{matrix}
0.15 & 0.25 \\
0.2 & 0.3 \\
0.35 & 0.35
\end{matrix}
\right]
\end{align}

<br/>

\begin{align}
\textrm{Weights to Output Layer: }
\left[
\begin{matrix}
0.4 & 0.5 \\
0.45 & 0.55 \\
0.6 & 0.6
\end{matrix}
\right]
\end{align}

<br/>

\begin{align}
\textrm{Learning Rate: } 0.5
\end{align}

<br/>


In [1]:
import numpy as np

In [2]:
# inputs array
inputArr=[0.05,0.1]

# outputs array
outputArr=[0.01,0.99]

#learning rate
lr=0.5

In [3]:
class NeuralNetworkForwardPass():
  """
  computes single forward step of neural network
  """
  def __init__(self,inputs,outputs):
    self.inputs = inputs
    self.inputsWithBias = np.array([*self.inputs,1])
    self.outputs = outputs
    self.weightsToHl=np.array([[.15,0.25],[0.2,0.3],[0.35,0.35]])
    self.weightsToOl=np.array([[.4,0.5],[0.45,0.55],[0.6,0.6]])

  ### Example uses the sigmoid activation function for all layers
  def sigmoid(self,x,derivative=False):
    if derivative:
      return x*(1-x)
    return 1/(1+np.exp(-x))

  def getHlOutputs(self):    
    self.inputsWithBias = np.array([*self.inputs,1])
    #multiply 1x3 input array with 3x2 weights array
    netHl = self.inputsWithBias @ self.weightsToHl
    return self.sigmoid(x=netHl,derivative=False)

  def forwardPass(self):
    hlOutputs=self.getHlOutputs()
    self.hlOutputsWithBias=np.array([*hlOutputs,1])
    netOl= self.hlOutputsWithBias @ self.weightsToOl
    # actFn=self.sigmoid
    return self.sigmoid(x=netOl,derivative=False)

network=NeuralNetworkForwardPass(inputArr,outputArr)

olOutputs=network.forwardPass()
print(olOutputs)

[0.75136507 0.77292847]


<h3 align='center'>backward pass follows the chain rule</h3>

\begin{align}
\frac{\partial x}{\partial y}=\frac{\partial x}{\partial u}\frac{\partial u}{\partial y}
\end{align}

<br/>

<center><img src='https://github.com/snsie/ai-webdev/blob/main/images/matt_backward_output_node.png?raw=true'/></center>

<br/>

---

<br/>

<center><img src='https://github.com/snsie/ai-webdev/blob/main/images/matt_backward_network.png?raw=true'/></center>

<br/>


###<center>Equation to calculate new weights</center>
<br/>

\begin{align}
w_{t+1} =w_t - \eta\;\cdot\;\frac{\partial E_{total}}{\partial w_t} 
\end{align}
<br/>


In [123]:
class NeuralNetworkBackwardPass(NeuralNetworkForwardPass):
    """
    updates weights via backpropagation
    """
    def __init__(self,inputs,outputs, lr):
        super().__init__(inputs,outputs)
        # predicted from forward pass outputs
        self.lr=lr
        self.hlOutputs=self.getHlOutputs()
        self.predOutputs = self.forwardPass()
        # print(self.predOutputs)
    
    # get the change in total error with respect to each ol output
    def get_dE_dOlOut(self):
        return self.predOutputs-self.outputs
    
    # get the change in each ol node with respect to the net sum into each respective ol node
    def get_dOlOut_dOlNet(self):
      return self.sigmoid(self.predOutputs,derivative=True)

    def get_dOlNet_dWOl(self):
      return self.hlOutputsWithBias.reshape(len(self.hlOutputsWithBias),1)

    def get_dE_dOlNet(self):
      dE_dOlOut=self.get_dE_dOlOut()
      dOlOut_dOlNet=self.get_dOlOut_dOlNet()
      dE_dOlNet=dE_dOlOut*dOlOut_dOlNet
      return dE_dOlNet

    def getPartialWeightsToOl(self):
      dE_dOlNet=self.get_dE_dOlNet()
      dE_dOlNet=dE_dOlNet.reshape(1,len(dE_dOlNet))

      dOlNet_dWOl=self.get_dOlNet_dWOl()
      return dOlNet_dWOl@dE_dOlNet

    def getNextWeightsToOl(self):
      # get_dE_dOlOut=self.get_dE_dOlOut()
      partialWeightsToOl=self.getPartialWeightsToOl()
      # print(self.weightsToOl.shape)
      nextWeightsToOl=self.weightsToOl-self.lr*partialWeightsToOl
      return nextWeightsToOl

    ########## updating weights to hidden layer below

    def get_dE_dHlOut(self):
      dE_dOlNet=self.get_dE_dOlNet()
      # print(np.size(dE_dOlNet))
      dE_dOlNet=dE_dOlNet.reshape(np.size(dE_dOlNet),1)
      weightsToOlWithoutBias=np.delete(self.weightsToOl,2,0)
      dOlNet_dHlOut=weightsToOlWithoutBias
      return dOlNet_dHlOut@dE_dOlNet
    
    def get_dHlOut_dHlNet(self):
      return self.sigmoid(self.hlOutputs,derivative=True)

    def get_dE_dHlNet(self):
      dE_dHlOut=self.get_dE_dHlOut().flatten()
      dHlOut_dHlNet=self.get_dHlOut_dHlNet()
      return dE_dHlOut*dHlOut_dHlNet

    def get_dHlNet_dWHl(self):
      return self.inputsWithBias.reshape(len(self.inputsWithBias),1)

    def getPartialWeightsToHl(self):
      dE_dHlNet=self.get_dE_dHlNet()
      dE_dHlNet=dE_dHlNet.reshape(1,len(dE_dHlNet))
      dHlNet_dWHl=self.get_dHlNet_dWHl()
      return dHlNet_dWHl@dE_dHlNet

    def getNextWeightsToHl(self):
      partialWeightsToHl=self.getPartialWeightsToHl()
      nextWeightsToHl=self.weightsToHl-self.lr*partialWeightsToHl
      return nextWeightsToHl

net=NeuralNetworkBackwardPass(inputArr,outputArr,lr)
net.getNextWeightsToHl()

[[0.14978072 0.24975114]
 [0.19956143 0.29950229]
 [0.34561432 0.34502287]]
